In [7]:
from datetime import datetime, date
from itertools import chain
from typing import cast

import torch
from elasticai.creator.nn.fixed_point._math_operations import MathOperations
from elasticai.creator.file_generation.on_disk_path import OnDiskPath
from elasticai.creator.nn.fixed_point._two_complement_fixed_point_config import FixedPointConfig
from elasticai.creator.vhdl.code_generation.code_abstractions import to_vhdl_binary_string
from elasticai.creator.vhdl.testbench_helper import tensor_to_vhdl_vector
from torch import nn, Tensor

# Creator imports
from elasticai.creator.nn import Sequential
from elasticai.creator.nn.fixed_point import Linear, Tanh
from package.dnn.pytorch_handler import __model_settings_common
from package.dnn.pytorch_handler import ModelRegistry

In [8]:
models_available = ModelRegistry()

In [9]:
total_bits = 8
frac_bits = 5
bn_affine = True
num_steps = 2**8
interval = (-4, 3.96875)

In [10]:
@models_available.register
class dnn_ae_v1_Q_wo_BN(__model_settings_common):
    """Class of an autoencoder with Dense-Layer for feature extraction"""
    def __init__(self, input_size=32, output_size=3):
        super().__init__('Autoencoder')
        self.model_shape = (1, input_size)
        self.model_embedded = True
        iohiddenlayer = [input_size, 20, 14, output_size]
        do_train_bias = True
        do_train_batch = True

        # --- Encoder Path
        self.encoder = Sequential(
            Linear(in_features=iohiddenlayer[0], out_features=iohiddenlayer[1], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[1], out_features=iohiddenlayer[2], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[2], out_features=iohiddenlayer[3], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
        )
        # --- Decoder Path
        self.decoder = Sequential(
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps),
            Linear(in_features=iohiddenlayer[3], out_features=iohiddenlayer[2], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[2], out_features=iohiddenlayer[1], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
            Tanh(total_bits=total_bits, frac_bits=frac_bits, num_steps=num_steps, sampling_intervall=interval),
            Linear(in_features=iohiddenlayer[1], out_features=iohiddenlayer[0], bias=do_train_bias, total_bits=total_bits, frac_bits=frac_bits, parallel=False),
        )

    def forward(self, x: Tensor) -> [Tensor, Tensor]:
        encoded = self.encoder(x)
        return encoded, self.decoder(encoded)
    
    def forward_first_layer(self, x: Tensor) -> Tensor:
        return self.encoder[0](x)
    
    def create_design(self, name):
        encoder = self.encoder.create_design(f"{name}_encoder")
        decoder = self.decoder.create_design(f"{name}_decoder")
        return encoder, decoder

In [11]:
autoencoder = torch.load(f="/home/silas/PycharmProjects/denspp.offline/3_Python/runs/20250127_122436_train_dnn_ae_v1_Q_wo_BN_ae/model_ae_fold000_epoch0028.pth")

/tmp/ipykernel_8605/535421855.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoencoder = torch.load(f="/home/silas/PycharmProjects/denspp.offline/3_Python/runs/202501

In [12]:
path4vhdl = f'vhdl/run_{date.today()}_{datetime.utcnow().strftime("%H%M")}'

destination = OnDiskPath(path4vhdl)
encoder, decoder = autoencoder.create_design("autoencoder")
encoder.save_to(destination)